<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb -qU
!wandb login 617d80dc70f383d8ff9ef1c2b94a78ccc8d8db23

     |████████████████████████████████| 1.9 MB 31.5 MB/s 
     |████████████████████████████████| 166 kB 69.1 MB/s 
     |████████████████████████████████| 182 kB 70.3 MB/s 
     |████████████████████████████████| 63 kB 831 kB/s 
     |████████████████████████████████| 166 kB 72.7 MB/s 
     |████████████████████████████████| 162 kB 76.0 MB/s 
     |████████████████████████████████| 162 kB 79.9 MB/s 
     |████████████████████████████████| 158 kB 73.6 MB/s 
     |████████████████████████████████| 157 kB 77.0 MB/s 
     |████████████████████████████████| 157 kB 76.1 MB/s 
     |████████████████████████████████| 157 kB 78.6 MB/s 
     |████████████████████████████████| 157 kB 75.0 MB/s 
     |████████████████████████████████| 157 kB 76.3 MB/s 
     |████████████████████████████████| 157 kB 70.3 MB/s 
     |████████████████████████████████| 157 kB 77.1 MB/s 
     |████████████████████████████████| 156 kB 79.5 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [ ]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [ ]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [ ]:
mobilenet = tf.keras.applications.MobileNetV2(
    input_shape=None,
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)
model_mobilenet=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  mobilenet,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
mobilenet.trainable=False
model_mobilenet.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1000)             3538984   
 ional)                                                          
                                                                 
 dropout_1 (Dropout)         (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 5)                 5005      
                                                                 
Total params: 3,543,989
Trainable params: 5,005
Non-trainable params: 3,538,984
_________________________________________________________________


In [ ]:
model_mobilenet.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [ ]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "MobilenetV2",
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [ ]:
%%time
epochs=40
history = model_mobilenet.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.5934 - accuracy: 0.4886 

wandb: Adding directory to artifact (/content/wandb/run-20221106_041221-9izj1zp4/files/model-best)... Done. 0.1s


22/22 [==============================] - 287s 9s/step - loss: 1.5934 - accuracy: 0.4886 - val_loss: 1.5824 - val_accuracy: 0.5600
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 1.5679 - accuracy: 0.6562

wandb: Adding directory to artifact (/content/wandb/run-20221106_041221-9izj1zp4/files/model-best)... Done. 0.2s


...

22/22 [==============================] - 21s 959ms/step - loss: 1.1319 - accuracy: 0.6861 - val_loss: 1.1695 - val_accuracy: 0.6686
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 1.1282 - accuracy: 0.6747

wandb: Adding directory to artifact (/content/wandb/run-20221106_041221-9izj1zp4/files/model-best)... Done. 0.2s


22/22 [==============================] - 21s 970ms/step - loss: 1.1282 - accuracy: 0.6747 - val_loss: 1.1637 - val_accuracy: 0.6686
Epoch 40/40
22/22 [==============================] - ETA: 0s - loss: 1.1149 - accuracy: 0.6818

wandb: Adding directory to artifact (/content/wandb/run-20221106_041221-9izj1zp4/files/model-best)... Done. 0.1s


22/22 [==============================] - 21s 975ms/step - loss: 1.1149 - accuracy: 0.6818 - val_loss: 1.1578 - val_accuracy: 0.6686
CPU times: user 13min 26s, sys: 39.6 s, total: 14min 5s
Wall time: 25min 51s


In [ ]:
wandb.finish()

accuracy,▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇██▇▇█▇▇▇████▇█▇████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁
val_accuracy,▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████
val_loss,██▇▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
accuracy,0.68182
best_epoch,39
best_val_loss,1.15778
epoch,39
loss,1.11495
val_accuracy,0.66857


In [ ]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
